In [42]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
 
i = 2
df_acc = pd.read_csv(f'./Accelerometer ({i}).csv')
df_acc.loc[:,'time'] = (df_acc.loc[:,'time'] - df_acc.loc[0,'time']) / 1000000000

In [12]:
def kalman_filter(len_data, data):
    Q = 0.001  # process variance
    R = 0.5  # measurement variance
    filtered = np.zeros(len_data)  # initial state estimate
    P = np.zeros(len_data)  # initial state covariance
    K = np.zeros(len_data)  # Kalman gain
    for k in range(len_data):
        filtered[k] = filtered[k - 1]
        P[k] = P[k - 1] + Q
        K[k] = P[k] / (P[k] + R)
        filtered[k] = filtered[k] + K[k] * (data[k] - filtered[k])
        P[k] = (1 - K[k]) * P[k]
    return filtered

In [44]:
len_arr = df_acc.loc[:, 'time'].shape[0]
dt = (df_acc.loc[:,'time'][2] - df_acc.loc[:,'time'][1] + df_acc.loc[:,'time'][4] - df_acc.loc[:,'time'][3])/2

x_filtered = kalman_filter(len_arr, df_acc.loc[:, 'x'])
y_filtered = kalman_filter(len_arr, df_acc.loc[:, 'y'])
fig_x = go.Figure()
fig_y = go.Figure()
fig_z = go.Figure()
 
fig_x.add_trace(go.Scatter(x = df_acc.loc[:,'time'], y = df_acc.loc[:, 'x'], name = 'a_x', line = dict(width=0.5), line_color='red'))
fig_x.add_trace(go.Scatter(x = df_acc.loc[:,'time'], y = np.cumsum(x_filtered*0.0098801), name = 'filtered a_x', line_color='blue'))
fig_y.add_trace(go.Scatter(x = df_acc.loc[:,'time'], y = df_acc.loc[:, 'y'], name = 'a_y', line = dict(width=0.5), line_color='red'))
fig_y.add_trace(go.Scatter(x = df_acc.loc[:,'time'], y = np.cumsum(y_filtered*0.0098801), name = 'filtered a_y', line_color='blue')) 
fig_z.add_trace(go.Scatter(x = df_acc.loc[:,'time'], y = df_acc.loc[:, 'z'], name = 'a_z', line = dict(width=0.5), line_color='red'))
fig_z.add_trace(go.Scatter(x = df_acc.loc[:,'time'], y = np.cumsum(y_filtered*0.0098801), name = 'filtered a_z', line_color='blue')) 

fig_x.update_layout(xaxis_title="time, s", yaxis_title="a, m/s^2")
fig_y.update_layout(xaxis_title="time, s", yaxis_title="a, m/s^2")
fig_z.update_layout(xaxis_title="time, s", yaxis_title="a, m/s^2")
 
fig_x.show()
fig_y.show()
fig_z.show()


In [36]:
a0 = 1.61
a1 = 1.7
a2 = 1.62
a4 = 1.55
a5 = 1.72
a_all = np.array([a0, a1, a2, a4, a5])
np.mean(a_all) * 3.6

5.904